In [1]:
import requests
from collections import defaultdict
from datetime import datetime, timedelta
from langchain.agents import Tool
from langchain.llms import HuggingFacePipeline
from langchain.agents import initialize_agent
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

/home/vatsal-patel/anaconda3/envs/unsloth_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def get_forecast(city):
    """Fetches 5-day weather forecast for a city using OpenWeatherMap API."""
    url_forecast = f"http://api.openweathermap.org/data/2.5/forecast?q={city}&appid={api_key}&units=metric"
    response_forecast = requests.get(url_forecast)
    if response_forecast.status_code != 200:
        return "Error: Could not fetch forecast data."
    forecast_json = response_forecast.json()
    return restructure_forecast(forecast_json)

def restructure_forecast(forecast_json):
    """Restructures the forecast JSON data into a nested dictionary by date and time."""
    structured_data = defaultdict(dict)
    for entry in forecast_json['list']:
        date, time = entry['dt_txt'].split()
        structured_data[date][time] = {
            'temperature': entry['main']['temp'],
            'feels_like': entry['main']['feels_like'],
            'temp_min': entry['main']['temp_min'],
            'temp_max': entry['main']['temp_max'],
            'pressure': entry['main']['pressure'],
            'humidity': entry['main']['humidity'],
            'weather': entry['weather'][0]['description'],
            'icon': entry['weather'][0]['icon'],
            'wind_speed': entry['wind']['speed'],
            'wind_deg': entry['wind']['deg'],
            'visibility': entry['visibility'],
            'pop': entry['pop'],
            'rain': entry.get('rain', {}).get('3h', 0),
            'clouds': entry['clouds']['all']
        }
    return structured_data


In [3]:
def get_specific_forecast(forecast_data, query):
    """
    Extracts specific forecast information based on user queries.
    
    Args:
    forecast_data (dict): The structured forecast data.
    query (str): The user's query.
    
    Returns:
    str: The relevant forecast information.
    """
    today = datetime.today()
    query = query.lower()

    if 'tomorrow' in query:
        target_date = (today + timedelta(days=1)).strftime('%Y-%m-%d')
    elif 'day after tomorrow' in query:
        target_date = (today + timedelta(days=2)).strftime('%Y-%m-%d')
    else:
        # Extract specific date if mentioned in the query
        try:
            target_date = datetime.strptime(query.split()[-1], '%Y-%m-%d').strftime('%Y-%m-%d')
        except ValueError:
            # Default to today's date if no specific date is mentioned
            target_date = today.strftime('%Y-%m-%d')

    if 'morning' in query:
        target_time = '09:00:00'
    elif 'afternoon' in query:
        target_time = '15:00:00'
    elif 'evening' in query:
        target_time = '18:00:00'
    elif 'night' in query:
        target_time = '21:00:00'
    else:
        # Default to the next available time if no specific time is mentioned
        target_time = '12:00:00'

    if target_date in forecast_data:
        if target_time in forecast_data[target_date]:
            forecast = forecast_data[target_date][target_time]
            return (f"Forecast for {target_date} at {target_time}:\n"
                    f"Temperature: {forecast['temperature']}°C\n"
                    f"Feels like: {forecast['feels_like']}°C\n"
                    f"Weather: {forecast['weather']}\n"
                    f"Humidity: {forecast['humidity']}%\n"
                    f"Wind: {forecast['wind_speed']} m/s, {forecast['wind_deg']}°\n"
                    f"Visibility: {forecast['visibility']} meters\n"
                    f"Probability of precipitation: {forecast['pop']*100}%\n"
                    f"Rain: {forecast['rain']} mm\n"
                    f"Clouds: {forecast['clouds']}%")
        else:
            return f"No forecast data available for {target_date} at {target_time}."
    else:
        return f"No forecast data available for {target_date}."

def get_forecast_info(city, query):
    """Fetches and processes the weather forecast based on the user's query."""
    forecast_data = get_forecast(city)
    if isinstance(forecast_data, dict):
        return get_specific_forecast(forecast_data, query)
    else:
        return forecast_data

In [5]:
from langchain.llms import LlamaCpp

In [9]:
llm = LlamaCpp(model_path="../models/Phi-3-small-8k-instruct/", n_gpu_layers=0) 

llama_model_load: error loading model: llama_model_loader: failed to load model from ../models/Phi-3-small-8k-instruct/

gguf_init_from_file: invalid magic characters '7s  '
llama_load_model_from_file: failed to load model


ValidationError: 1 validation error for LlamaCpp
__root__
  Could not load Llama model from path: ../models/Phi-3-small-8k-instruct/. Received error Failed to load model from file: ../models/Phi-3-small-8k-instruct/ (type=value_error)

In [4]:
model = AutoModelForCausalLM.from_pretrained(
    "../models/unsloth/Phi-3-mini-4k-instruct/",  
    device_map="cuda",  
    torch_dtype="auto",  
    trust_remote_code=True
)

Loading checkpoint shards:   0%|                                                                                                                                        | 0/2 [00:02<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 18.00 MiB. GPU 

In [2]:




# Forecast Tool
forecast_tool = Tool(
    name="ForecastLookup",
    func=lambda city: get_forecast_info(city, "Please specify a query."),
    description="Useful to get the 5-day weather forecast for a city."
)


# Example Usage
# print(agent.run("What's the weather like in New York?"))
print(agent.run("What's the forecast for the next 5 days in Kalyani?"))

# Example Usage with a specific query
print(agent.run("What's the forecast for tomorrow morning in London?"))


A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-128k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-128k-instruct:
- modeling_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.

KeyboardInterrupt

